In [ ]:
# Instalação
* Oracle over Python: https://www.oracle.com/database/technologies/appdev/python/quickstartpythononprem.html#linux-tab
* Download **Instant Client Basic Light**:
    - Linux: https://download.oracle.com/otn_software/linux/instantclient/oracle-instantclient-basiclite-linuxx64.rpm
    - Windows: https://www.oracle.com/database/technologies/instant-client/winx64-64-downloads.html 
    - MacOS: https://www.oracle.com/database/technologies/instant-client/macos-intel-x86-downloads.html
* Linux:
    * sudo alien -i oracle-instantclient-basiclite-linuxx64.rpm
   

### Fazendo a instalação dos pacotes necessários

In [1]:
!pip install cx_Oracle

### Carregar as bibliotecas

In [2]:
import cx_Oracle
import pandas as pd
import sqlalchemy

In [3]:
#Howto: https://www.oracle.com/database/technologies/appdev/python/quickstartpythononprem.html#copy
#executar uma vez por sessão - lib dir é o diretório onde se encontra o arquivo libclntsh.so

# Indicar o local onde o cx_Oracle pode encontrar os arquivos do client do Oracle que instalou anteriormente
try:
    cx_Oracle.init_oracle_client(lib_dir=r"D:\Oracle\instantclient_19_11")
except Exception as e:
    print(str(e))

### Conectar ao banco de dados Oracle

In [4]:
dsnStr = cx_Oracle.makedsn("localhost", "1521", "xe")
conexao = cx_Oracle.connect(user="system", password="password", dsn=dsnStr)
print(conexao.version)


18.4.0.0.0


In [ ]:
### SOMENTE SE QUISER USAR PostgreSQL

In [ ]:
############################################
#  SOMENTE PARA Postgresql
############################################
import psycopg2
conexao = psycopg2.connect(host='localhost', database='DB1', user='postgres', password='pgadmin')


import pandas.io.sql as psql
from sqlalchemy import create_engine

## Cursor
* Uma conexão fornece objetos do tipo `cursor`; os quais permitem operações DDL e DML no banco conectado.

In [5]:
# Abriremos um cursor para instruções DDL e DML
cursorDDL = conexao.cursor()
cursorDML = conexao.cursor()

## Criação de um esquema a partir de um script sql

    1) Aqui fazemos a leitura completa de um arquivo;
    2) Quebra do texto usando a operação de split, considerando o ";" como separador;
    3) Para cada comando encontrado, executamos o SQL por meio de um cursor;
    4) Tratamos eventuais erros usando uma construção try-except.

In [6]:
#Primeiro o esquema
fd = open('./Dados_Aula07/Esquema_Papelaria.sql', 'r')
sqlFile = fd.read()
fd.close()

sqlCommands = sqlFile.split(';')
for command in sqlCommands:
    if(command == ''): break;
    try:
        print(command)
        print()
        cursorDDL.execute(command)
    except Exception as msg:
        print("Erro de SQL: "+ str(msg) + ": "+ command)
print('Script finalizado.')
# Um erro de inválido SQL Statement é dado, mas por causa de FIM de arquivo. IGNORAR!

Drop table produto cascade constraints


Drop table vendas cascade constraints


Drop table funcionario cascade constraints

Erro de SQL: ORA-00942: a tabela ou view não existe: 
Drop table funcionario cascade constraints


CREATE TABLE produto (
      idproduto INTEGER NOT NULL,
      nome       VARCHAR2(40) NOT NULL,
      categoria     CHAR(15),
      preco DECIMAL(6, 2),
            
      CONSTRAINT pk_produto PRIMARY KEY (idproduto)
)



CREATE TABLE vendas (
      numpedido      INTEGER NOT NULL,
      codvendedor    INTEGER,
      idproduto      INTEGER,
      quantidade     INTEGER,

      CONSTRAINT pk_vendas PRIMARY KEY (numpedido)
)




CREATE TABLE funcionario (
      codvendedor     INTEGER,
      nome            VARCHAR2(40) NOT NULL,
           
      CONSTRAINT pk_jogador PRIMARY KEY (codvendedor))

Erro de SQL: ORA-02264: nome já usado por uma restrição existente: 


CREATE TABLE funcionario (
      codvendedor     INTEGER,
      nome            VARCHAR2(40) NOT NULL,

In [7]:
#Agora as relações instanciadas
fd = open('./Dados_Aula07/Dados_Papelaria.sql', 'r')
sqlFile = fd.read()
fd.close()

sqlCommands = sqlFile.split(';')
for command in sqlCommands:
    try:
        print(command)
        print()
        cursorDDL.execute(command)        
    except Exception as msg:
        print("Erro de SQL: "+ str(msg) + ": |"+ command+'|')
print('Script finalizado.')


insert into produto values (1,'Caneta Stabilo 04 azul', 'ambos', (17.90))


insert into produto values (2,'Caneta Stabilo 04 preta', 'ambos', (17.90))


insert into produto values (3,'Caneta Stabilo 04 verde', 'ambos', (17.90))


insert into produto values (4,'Caneta Stabilo 04 vermelha', 'ambos', (17.90))


insert into produto values (5,'Caneta Unipen 08', 'ambos', (22.90))


insert into produto values (6,'Caneta Unipen 05', 'ambos', (22.90))


insert into produto values (7,'Post-it MÃ©dio', 'escritÃ³rio', (8.10))


insert into produto values (8,'Compasso', 'escolar', (4.50))




insert into funcionario values (1,'Atadolfa')

Erro de SQL: ORA-00942: a tabela ou view não existe: |


insert into funcionario values (1,'Atadolfa')|

insert into funcionario values (2,'Neoclesina')

Erro de SQL: ORA-00942: a tabela ou view não existe: |
insert into funcionario values (2,'Neoclesina')|



insert into vendas values (1,2,8,2)


insert into vendas values (2,2,4,1)


insert into vendas values (

* Inserindo alguns dados.

In [8]:
cursorDML.execute("insert into produto values (9,'Lapiseira Pentel 07', 'ambos', (19.80))");

cursorDML.execute("insert into produto values (10,'grampos', 'escritório', (4.80))");


* Se foi tudo ok, executar `commit()`; do contrário, executar `rollback()` para desfazer as atualizações

In [9]:
conexao.commit()

# Recuperando dados com cursores
* Pode-se requisitar os dados da base um por vez, o que consome mais recursos de rede e processamento no servidor; ou múltiplos por vez (batch), o que é mais eficiente. Se houver memória (ou poucos dados) pode-se recuperar todos os dados de uma vez.

* Um por vez

In [ ]:
cursorDML.execute("select * from produto where categoria = \'escolar\'")
while True:
    row = cursorDML.fetchone()  #Um por vez
    if row is None: break
    print(row)

* **Batch**: menos requisições de rede/transferência de dados; menos processamento no servidor

In [ ]:
cursorDML.execute("select * from produto")
num_rows = 3
while True:
    rows = cursorDML.fetchmany(num_rows) #Muitos por vez
    if not rows: break
    for row in rows:
        print(row)
        
print()
print("Foram recuperadas "+str(cursorDML.rowcount) +" tuplas.")


* É possível ainda recuperar meta-informações sobre as colunas retornadas por meio do cursor

In [ ]:
for metadata in cursorDML.description:
    print(metadata)

# Usando rowfactories

* Com rowfactories podemos processar os dados recuperados antes de retorná-los.

In [ ]:
cursorDML.execute("select * from vendas")
columns = [col[0] for col in cursorDML.description] # array com os nomes dos atributos

cursorDML.rowfactory = lambda *args: zip(columns, args) #aplica lambda para cada tupla


In [ ]:
#a operação de fetch garante a execução do lambda
data = cursorDML.fetchone()
#o resultado é o produto cartesiano {nomes do atributos} x {valores dos atributos}
print(list(data))

# Lendo dados do banco diretamente para um Pandas Dataframe

* Uma consulta SQL.

In [ ]:
meuDataFrame = pd.read_sql_query('SELECT * FROM produto', con=conexao)

In [ ]:
meuDataFrame

* ATENCAO: um dataframe nada mais é do que `uma tabela (uma relação) em memória`. Muito provavelmente, ler uma tabela inteira em memória vai causar problemas de falta de memória e de processamento. Uma solução é ler pedaços (**chunks**) da tabela em um objeto do tipo **generator**;

* Pode-se fazer isso usando-se uma variação do `read_sql_query()` com o parâmetro `chunksize` fornecido. Neste caso, o resultado não será um DataFrame, mas sim um objeto do tipo **generator**, o qual fornecerá um DataFrame com chunksize elementos a cada iteração;

* **Generator**: uma abstração de dados que permite a geração de resultados/produtos sequencialmente, os quais são produzidos mediante requisição. Após um generator ser consumido, ele precisa ser regerado.

In [ ]:
dataFrameGenerator = pd.read_sql_query('SELECT * FROM produto', con=conexao, chunksize=3)
type(dataFrameGenerator)

In [ ]:
for i, dataFrameChunk in enumerate(dataFrameGenerator):
    print('-'*10)
    print("Chunk "+str(i))
    print(dataFrameChunk)


* Com o processamento em chunks é possível executar processamento pedaço por pedaço do dataset.

In [ ]:
total_precos = 0
dataFrameGenerator = pd.read_sql_query('SELECT * FROM produto', con=conexao, chunksize=3)

In [ ]:
for i, dataFrameChunk in enumerate(dataFrameGenerator):
    total_chunk = dataFrameChunk['PRECO'].sum()
    print('Total de precos ' + str(i) + '-esimo chunk: '+str(total_chunk))
    total_precos += total_chunk
print("Total de precos: " + str(total_precos))

* ATENCAO: o processamento de agregações é muito eficiente em SGBDs relacionais. Trazer dados do database para a memória, computar, e depois totalizar só é recomendado para tarefas que não podem, ou que são muito complexas, em um SGBD. Para o exemplo da totalização, basta um simples SQL:

In [ ]:
for row in cursorDML.execute("Select SUM(quantidade) from vendas"): print(row[0])